<h1>Data Science Pipeline Project - Andrey Olv<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Convertendo

https://servicodados.ibge.gov.br/api/docs/localidades?versao=1

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15761-areas-dos-municipios.html?=&t=downloads

https://dados.gov.br/dataset/estacoes-licenciadas-a-operar-no-servico-movel-pessoal

https://dados.gov.br/dataset/estacoes-licenciadas-a-operar-no-servico-movel-pessoal/resource/b759bfe5-d820-48b0-88b8-7d191862225c

In [1]:
# pip install simplekml

In [16]:
import pandas as pd
import simplekml

Sem definir o encondig, as palavras com acentos não apareciam, ficava bugado. Com o encondig Latino, este problema foi resolvido. Sem definir o separador também estava dando erro. Defini o número de linhas pois a planilha é muito grande.

In [17]:
df = pd.read_csv("SLP (Serviço Limitado Privado).csv", sep=';', nrows = 10000, encoding='UTF-8')

Visualizando os dados importados

In [18]:
df.head(50)
df.shape
df.dtypes

CNPJ/CPF                        object
Tipo de Identificação           object
Nome/Razão Social               object
Nome do Serviço                 object
Aplicação do Serviço            object
Número da Estação                int64
UF                              object
Município                       object
Bairro                          object
Logradouro                      object
Latitude                        object
Longitude                       object
Altitude                        object
Data Inclusão                   object
Data Primeiro Licenciamento     object
Data Último Licenciamento       object
Data Validade Licença           object
Freq. Transmissão               object
Freq. Rexepção                  object
Designação Emissão              object
Potência                        object
Raio                            object
Ganho                           object
F/C                            float64
1/2 Potência                    object
Ângulo de Elevação       

In [6]:
df = df.drop(columns=['Bairro', 'Logradouro','Data Inclusão', 'Data Primeiro Licenciamento',
                'Designação Emissão', 'Potência', 'Raio', 'Ganho', 'F/C', '1/2 Potência', 'UF',
                'Ângulo de Elevação', 'Azimute', 'Polarização', 'Data Último Licenciamento',
                'Aplicação do Serviço', 'Tipo de Identificação', 'Altitude', 'Freq. Rexepção'])
                
df = df.rename(columns={"CNPJ/CPF": "CNPJ", "Nome/Razão Social": "Empresa", "Freq. Transmissão": "Frequência"})

df.dtypes
df.head()

,CNPJ,Empresa,Nome do Serviço,Número da Estação,Município,Latitude,Longitude,Data Validade Licença,Frequência,Altura
0,39819743000123,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,618764,Ibiraçu,19S490000,40W210000,26/08/2014,"170,67000000 MHz",20
1,39819743000123,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,618764,Ibiraçu,19S490000,40W210000,26/08/2014,"170,89000000 MHz",20
2,39819743000123,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,96792,Vila Velha,20S193300,40W163400,26/08/2014,"162,79000000 MHz",20
3,39819743000123,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,96792,Vila Velha,20S193300,40W163400,26/08/2014,"163,73000000 MHz",20
4,39819743000123,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,96792,Vila Velha,20S193300,40W163400,26/08/2014,"163,89000000 MHz",20


Decimal degrees = Degrees + (Minutes/60) + (Seconds/3600)

For Example, to convert 39° 25' 30" to decimal degrees

First, convert minutes and seconds to their degree equivalents and add the results
25'/60 = 0.4167°
30"/3600 = .0083°
and 0.4167° + 0.0083° = 0.425°
Then, add this number to the number of degrees.
39° + 0.425° = 39.425°
So, the final result is:
39° 25' 30" = 39.425°

In [7]:
df.Latitude

0       19S490000
1       19S490000
2       20S193300
3       20S193300
4       20S193300
          ...    
9995    22S513185
9996    22S513185
9997    22S513185
9998    22S513185
9999    22S513185
Name: Latitude, Length: 10000, dtype: object

In [8]:
df.Latitude.dropna

<bound method Series.dropna of 0       19S490000
1       19S490000
2       20S193300
3       20S193300
4       20S193300
          ...    
9995    22S513185
9996    22S513185
9997    22S513185
9998    22S513185
9999    22S513185
Name: Latitude, Length: 10000, dtype: object>

In [9]:
def convert(coordenada):
    try:
        degree = coordenada[:2]
        minutes = coordenada[3:5]
        seconds = coordenada[5:]
        letra = coordenada[2]

        if letra == 'W' or letra == 'S':
            signal = -1

        if letra == 'O' or letra == 'N':
            signal = 1

        decimal_degrees = int(degree) + (int(minutes)/60) + (int(seconds)/360000)
        final = signal*decimal_degrees

        return final

    except IndexError:
        return 0

In [10]:
df['Latitude'] = df.Latitude.apply(convert)
df['Latitude']

0      -19.816667
1      -19.816667
2      -20.325833
3      -20.325833
4      -20.325833
          ...    
9995   -22.858847
9996   -22.858847
9997   -22.858847
9998   -22.858847
9999   -22.858847
Name: Latitude, Length: 10000, dtype: float64

In [22]:
df['NewLatitude'] = df.Latitude.apply(convert)
df['NewLatitude']

0      -19.816667
1      -19.816667
2      -20.325833
3      -20.325833
4      -20.325833
          ...    
9995   -22.858847
9996   -22.858847
9997   -22.858847
9998   -22.858847
9999   -22.858847
Name: NewLatitude, Length: 10000, dtype: float64

In [23]:
df['NewLongitude'] = df.Longitude.apply(convert)
df['NewLongitude']

0      -40.350000
1      -40.350000
2      -40.276111
3      -40.276111
4      -40.276111
          ...    
9995   -47.045308
9996   -47.045308
9997   -47.045308
9998   -47.045308
9999   -47.045308
Name: NewLongitude, Length: 10000, dtype: float64

In [29]:
df[["Latitude", "NewLatitude","Longitude", "NewLongitude"]].head(50)

,Latitude,NewLatitude,Longitude,NewLongitude
0,19S490000,-19.816667,40W210000,-40.350000
1,19S490000,-19.816667,40W210000,-40.350000
2,20S193300,-20.325833,40W163400,-40.276111
3,20S193300,-20.325833,40W163400,-40.276111
4,20S193300,-20.325833,40W163400,-40.276111
5,20S193300,-20.325833,40W163400,-40.276111
6,20S193300,-20.325833,40W163400,-40.276111
7,20S193300,-20.325833,40W163400,-40.276111
8,20S193300,-20.325833,40W163400,-40.276111
9,20S193300,-20.325833,40W163400,-40.276111


In [8]:
df.shape

(10000, 29)

In [13]:
df.dtypes

CNPJ                      object
Empresa                   object
Nome do Serviço           object
Número da Estação          int64
Município                 object
Latitude                 float64
Longitude                 object
Data Validade Licença     object
Frequência                object
Altura                    object
dtype: object

In [15]:
df = df.drop(columns=['Bairro', 'Logradouro','Data Inclusão', 'Data Primeiro Licenciamento',
                'Designação Emissão', 'Potência', 'Raio', 'Ganho', 'F/C', '1/2 Potência',
                'Ângulo de Elevação', 'Azimute', 'Polarização', 'Data Último Licenciamento',
                'Aplicação do Serviço', 'Tipo de Identificação'])
                
df.rename(columns={"CNPJ/CPF": "CNPJ", "Nome/Razão Social": "Empresa"})

df.dtypes


KeyError: "['Bairro' 'Logradouro' 'Data Inclusão' 'Data Primeiro Licenciamento'\n 'Designação Emissão' 'Potência' 'Raio' 'Ganho' 'F/C' '1/2 Potência'\n 'Ângulo de Elevação' 'Azimute' 'Polarização' 'Data Último Licenciamento'\n 'Aplicação do Serviço' 'Tipo de Identificação'] not found in axis"

In [31]:
# df['Latitude'] = pd.to_numeric(df['Latitude'])
# df['Longitude'] = pd.to_numeric(df['Longitude'])

In [33]:
df.head(20)

,CNPJ/CPF,Tipo de Identificação,Nome/Razão Social,Nome do Serviço,Aplicação do Serviço,Número da Estação,UF,Município,Bairro,Logradouro,...,Raio,Ganho,F/C,1/2 Potência,Ângulo de Elevação,Azimute,Polarização,Altura,NewLatitude,NewLongitude
0,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,618764,ES,Ibiraçu,NaN,MORRO DE ARICANGA - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-19.816667,-40.350000
1,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,618764,ES,Ibiraçu,NaN,MORRO DE ARICANGA - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-19.816667,-40.350000
2,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
3,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
4,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
5,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
6,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
7,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
8,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96822,ES,Vila Velha,PRAIA DA COSTA,VIATURA OPERANDO A 50 KM DA BASE - nº .,...,"60,0","0,0",NaN,NaN,NaN,NaN,V,"2,0",-20.325833,-40.276111
9,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96822,ES,Vila Velha,PRAIA DA COSTA,VIATURA OPERANDO A 50 KM DA BASE - nº .,...,"60,0","0,0",NaN,NaN,NaN,NaN,V,"2,0",-20.325833,-40.276111


Eliminando as linhas com número de estação duplicada.

In [34]:
df["Número da Estação"].unique()
df_unique = df.drop_duplicates(subset = "Número da Estação")
df_unique

,CNPJ/CPF,Tipo de Identificação,Nome/Razão Social,Nome do Serviço,Aplicação do Serviço,Número da Estação,UF,Município,Bairro,Logradouro,...,Raio,Ganho,F/C,1/2 Potência,Ângulo de Elevação,Azimute,Polarização,Altura,NewLatitude,NewLongitude
0,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,618764,ES,Ibiraçu,NaN,MORRO DE ARICANGA - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-19.816667,-40.350000
2,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96792,ES,Vila Velha,PRAIA DA COSTA,MORRO DO MORENO - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-20.325833,-40.276111
8,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,96822,ES,Vila Velha,PRAIA DA COSTA,VIATURA OPERANDO A 50 KM DA BASE - nº .,...,"60,0","0,0",NaN,NaN,NaN,NaN,V,"2,0",-20.325833,-40.276111
10,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,618772,ES,Ibiraçu,NaN,MOVEL EM VIATURA - nº .,...,"60,0","0,0",NaN,NaN,NaN,NaN,V,NaN,-19.816667,-40.350000
11,39819743000123,CNPJ,SELECTIVE CALL COMUNICACOES LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,618810,ES,Linhares,NaN,MORRO DA PEDRA ROXA - nº .,...,"60,0","6,0",NaN,NaN,NaN,NaN,V,"20,0",-19.383333,-40.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9921,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1088971,SP,Socorro,SERROTE,PEDRA DA BELA VISTA - nº SN,...,"50,0","6,0",NaN,NaN,NaN,NaN,V,"40,0",-22.659903,-46.490489
9927,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1355880,SP,Campinas,JD SANTANA,"ROD CAMPINAS MOGI MIRIM - nº Km 2,5",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-22.858333,-47.045278
9948,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1088963,SP,Serra Negra,ZONA RURAL,ACESSO R. 14 DE JULHO - nº sn,...,"50,0","6,0",NaN,NaN,NaN,NaN,V,"25,0",-22.618953,-46.677033
9962,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1367404,SP,Paulínia,NaN,ROD EST PAULINIA COSMOPOLIS - nº Km 132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-22.750000,-47.158333


Definindo valores a serem pesquisados:

In [11]:
lat_search = -22.344725
long_search = -49.095136
raio_search = 50 # in Km

In [12]:
lat_search.type

AttributeError: 'float' object has no attribute 'type'

At the equator for longitude and for latitude anywhere, the following approximations are valid:

1° = 111 km  (or 60 nautical miles)
0.1° = 11.1 km
0.01° = 1.11 km (2 decimals, km accuracy)
0.001° = 111 m

Fazendo conversções de kilometros para latitude e longitude:

In [36]:
lat_south = lat_search - (raio_search/111)

lat_north = lat_search + (raio_search/111)

long_west = long_search - (raio_search/111)

long_east = long_search + (raio_search/111)

Testando as conversões, estão OK

In [39]:
print(lat_south)
print(lat_north)
print(long_west)
print(long_east)

-22.79517545045045
-21.89427454954955
-49.54558645045045
-48.64468554954954


Definindo os limites a serem filtrados para a pesquisa baseado nos valores definidos:

In [40]:
cond1 = df_unique['NewLatitude'] > lat_south
cond2 = df_unique['NewLatitude'] < lat_north
cond3 = df_unique['NewLongitude'] > long_west
cond4 = df_unique['NewLongitude'] < long_east

Filtrando o dataframe segundo as condições baseadas nos valores de pesquisa inseridos.

In [41]:
df_search = df_unique[cond1 & cond2 & cond3 & cond4]
df_search

,CNPJ/CPF,Tipo de Identificação,Nome/Razão Social,Nome do Serviço,Aplicação do Serviço,Número da Estação,UF,Município,Bairro,Logradouro,...,Raio,Ganho,F/C,1/2 Potência,Ângulo de Elevação,Azimute,Polarização,Altura,NewLatitude,NewLongitude
51,02756315000199,CNPJ,TRACKER DO BRASIL LTDA,Serviço Limitado Privado,011 - Limitado Privado - Prestação a terceiros,684579324,SP,Bauru,PARQUE ROOSEVELT,RUA MARIANO HERNANDES - nº 2-17,...,"43,0","9,0",NaN,NaN,NaN,NaN,V,"15,0",-22.303056,-49.101944
7433,47693643000121,CNPJ,DEPARTAMENTO AEROVIARIO DO ESTADO DE SAO PAULO,Serviço Limitado Privado,019 - Limitado Privado,684622335,SP,Bauru,JARDIM PLANALTO,RUA OTÁVIO PINHEIRO BRIZOLLA - nº 19/120,...,"2,0","1,7",NaN,NaN,NaN,NaN,V,"20,0",-22.347500,-49.056667
7434,47693643000121,CNPJ,DEPARTAMENTO AEROVIARIO DO ESTADO DE SAO PAULO,Serviço Limitado Privado,019 - Limitado Privado,684622343,SP,Bauru,JARDIM PLANALTO,RUA OTÁVIO PINHEIRO BRIZOLLA - nº 19/120,...,"2,0","1,7",NaN,NaN,NaN,NaN,V,"8,0",-22.347778,-49.052222
7435,47693643000121,CNPJ,DEPARTAMENTO AEROVIARIO DO ESTADO DE SAO PAULO,Serviço Limitado Privado,019 - Limitado Privado,684623439,SP,Bauru,NaN,MÓVEL VEICULAR - nº,...,"2,0","0,0",NaN,NaN,NaN,NaN,V,"2,0",-22.347500,-49.056667
7436,47693643000121,CNPJ,DEPARTAMENTO AEROVIARIO DO ESTADO DE SAO PAULO,Serviço Limitado Privado,019 - Limitado Privado,684627469,SP,Bauru,NaN,PORTATIL - nº,...,"2,0","0,0",NaN,NaN,NaN,NaN,V,"1,0",-22.347500,-49.056667
9490,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1092065,SP,Bauru,Z. RURAL,RODOVIA SP 294 - nº SN,...,"45,0","21,0","23,0","12,00","-0,29","21,6",H,"90,0",-22.288889,-49.098611
9504,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1092774,SP,Presidente Alves,CENTRO,RUA DR. CIRINO DE LIMA - nº 331,...,"50,0","6,0",NaN,NaN,NaN,NaN,V,"55,0",-22.100833,-49.435000
9705,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1004372148,SP,Macatuba,Centro,Rua Virgílio Ennei - nº 1000,...,"17,8","12,0","20,0","39,00","0,10","52,0",V,"10,0",-22.500100,-48.708589
9727,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1006075574,SP,Arealva,Zona Rural,Estrada Municipal Iacanaga-Arealva - nº SN,...,"42,6","12,0","20,0","48,00","0,10","109,6",V,"11,0",-22.021536,-48.923311
9730,33050196000188,CNPJ,COMPANHIA PAULISTA DE FORCA E LUZ,Serviço Limitado Privado,019 - Limitado Privado,1006075698,SP,Agudos,Zona Rural,Rodovia Marechal Rondon - nº Km 319,...,"16,9","12,0","20,0","48,00","0,10","129,9",V,"11,0",-22.456708,-48.972486


In [45]:
lat_list = df_search['NewLatitude'].values.tolist()
long_list = df_search['NewLongitude'].values.tolist()
name_list = df_search['Número da Estação'].values.tolist()

Definindo a função que cria o arquivo KML. A função já retorna a chamada para salvar o arquivo na diretório.

In [46]:
kml = simplekml.Kml()

def search_station():
    i = 0
    while i < len(lat_list):
        lat = lat_list[i]
        long = long_list[i]
        number = name_list[i]
        kml.newpoint(name=str(number), coords=[(long,lat)])  # lon, lat, optional height
        i = i+1
        
    return kml.save("EstaçõesPróximasSLP.kml")

In [47]:
search_station()

Aplicando a função para criar o arquivo KML